#1. Setup 

In [ ]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table
import pandas as pd

project = 'project name' # Project ID inserted based on the query results selected to explore
location = 'location' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

In [ ]:
%load_ext google.colab.data_table

#2. Celeb KPI of app use


-this month / last month action by time 

Chat (general message, reply, public message) / Like / Contents (photo, video, voice)

- create rate of increase and decrease




---




-이번 달 셀럽 활동 시간별

 (채팅 - 일반,답장,전체공개 포션 나눠서 / 좋아요 / 컨텐츠 - 사진동영상음성 )

 -지난 달 셀럽 활동 시간별

 (채팅 - 일반,답장,전체공개 포션 나눠서 / 좋아요 / 컨텐츠 - 사진동영상음성 )



`**-> Raw 이번 달 데이터 / 지난 달 대비 증감율 / 팬심 M 평균 / 팬심 M 평균 대비 **`








-이번 달 팬 활동 시간별

(채팅 / 좋아요 / 땡스 총액 / **접속시간** / 소속 채팅방)

-지난 달 팬 활동 시간별

(채팅 / 좋아요 / 땡스 총액 / **접속시간** / 소속 채팅방)




`**-> Raw 이번 달 데이터 / 지난 달 대비 증감율 / 팬심 M 평균 / 팬심 M 평균 대비 **`

#### 2.1. This month Chat data


identification columns are chat_pressedBtn_click and chat_page_btnicons_click


---


- 믹스패널 상에서는 chat_pressedBtn_click 과 chat_page_btnicons_click 에 나뉘어서 나옴



In [ ]:
%%bigquery celeb_chat --project  project name 

SELECT  user_nickname, user_type, message_length, chat_room_btn_icon_name, chat_celeb_nickname, chat_room_id, time

FROM `project name.mixpanel_test.mp_master_event` 

WHERE ((message_length > 0) OR (chat_room_btn_icon_name != ' ')) 
AND (time BETWEEN '2023-03-01 00:00:00' AND '2023-03-31 23:59:59')
AND (user_nickname = chat_celeb_nickname)

ORDER BY time

In [ ]:
celeb_chat_1 = celeb_chat.groupby(['chat_celeb_nickname', 'chat_room_id']).count()
celeb_chat_1 = celeb_chat_1.loc[:,['message_length','chat_room_btn_icon_name']]

celeb_chat_2 = pd.DataFrame(celeb_chat.groupby(['chat_celeb_nickname', 'chat_room_id'])['chat_room_btn_icon_name'].value_counts())
celeb_chat_2 = celeb_chat_2.rename(columns={'chat_room_btn_icon_name':'counts'})
celeb_chat_2 = celeb_chat_2.reset_index()


celeb_chat_2 = pd.pivot_table(celeb_chat_2,                # 피벗할 데이터프레임
                     index = ['chat_celeb_nickname','chat_room_id'],
                      values = 'counts',
                     columns = 'chat_room_btn_icon_name')    # 열 위치에 들어갈 열


celeb_chat_2 = celeb_chat_2.fillna(0)
celeb_chat_1 = celeb_chat_1.fillna(0)
celeb_chat_1 = celeb_chat_1.rename(columns={'chat_room_btn_icon_name':'total_btn_icon_name'})
celeb_chat_1 = celeb_chat_1.rename(columns={'message_length':'message'})

chat_celeb_total = pd.merge(celeb_chat_1, celeb_chat_2,left_index=True, right_index=True, how='outer')
chat_celeb_total = chat_celeb_total.fillna(0)


chat_celeb_total.rename(columns={'message':'일반채팅 수'}, inplace = True)
chat_celeb_total.rename(columns={'Reply':'답장 수'}, inplace = True)
chat_celeb_total.rename(columns={'Share':'전체공개 수'}, inplace = True)
chat_celeb_total.rename(columns={'Like':'좋아요 수'}, inplace = True)
chat_celeb_total['컨텐츠 수'] = chat_celeb_total['Image']+chat_celeb_total['Record']+chat_celeb_total['Video']

chat_celeb_total = chat_celeb_total.loc[: , ['일반채팅 수', '답장 수', '전체공개 수', '좋아요 수', '컨텐츠 수']]


celeb_chat_total_this_month = chat_celeb_total
celeb_chat_total_this_month

#### 2.1.  Last month Chat data


In [ ]:
%%bigquery celeb_chat_last_month --project  project name 

SELECT  user_nickname, user_type, message_length, chat_room_btn_icon_name, chat_celeb_nickname, chat_room_id, time

FROM `project name.mixpanel_test.mp_master_event` 

WHERE ((message_length > 0) OR (chat_room_btn_icon_name != ' ')) 
AND (time BETWEEN '2023-02-01 00:00:00' AND '2023-02-28 23:59:59')
AND (user_nickname = chat_celeb_nickname)

ORDER BY time

In [ ]:
celeb_chat_last_month

celeb_chat_1_last_month = celeb_chat_last_month.groupby(['chat_celeb_nickname', 'chat_room_id']).count()
celeb_chat_1_last_month = celeb_chat_1_last_month.loc[:,['message_length','chat_room_btn_icon_name']]

celeb_chat_2_last_month = pd.DataFrame(celeb_chat_last_month.groupby(['chat_celeb_nickname', 'chat_room_id'])['chat_room_btn_icon_name'].value_counts())
celeb_chat_2_last_month = celeb_chat_2_last_month.rename(columns={'chat_room_btn_icon_name':'counts'})
celeb_chat_2_last_month = celeb_chat_2_last_month.reset_index()

celeb_chat_2_last_month = pd.pivot_table(celeb_chat_2_last_month,               
                     index = ['chat_celeb_nickname','chat_room_id'],
                      values = 'counts',
                     columns = 'chat_room_btn_icon_name')    
                     

celeb_chat_2_last_month = celeb_chat_2_last_month.fillna(0)
celeb_chat_1_last_month = celeb_chat_1_last_month.fillna(0)
celeb_chat_1_last_month = celeb_chat_1_last_month.rename(columns={'chat_room_btn_icon_name':'total_btn_icon_name'})
celeb_chat_1_last_month = celeb_chat_1_last_month.rename(columns={'message_length':'message'})

celeb_chat_total_last_month = pd.merge(celeb_chat_1_last_month, celeb_chat_2_last_month,left_index=True, right_index=True, how='outer')
celeb_chat_total_last_month = celeb_chat_total_last_month.fillna(0)


celeb_chat_total_last_month.rename(columns={'message':'일반채팅 수'}, inplace = True)
celeb_chat_total_last_month.rename(columns={'Reply':'답장 수'}, inplace = True)
celeb_chat_total_last_month.rename(columns={'Share':'전체공개 수'}, inplace = True)
celeb_chat_total_last_month.rename(columns={'Like':'좋아요 수'}, inplace = True)
celeb_chat_total_last_month['컨텐츠 수'] = celeb_chat_total_last_month['Image']+celeb_chat_total_last_month['Record']+celeb_chat_total_last_month['Video']
celeb_chat_total_last_month = celeb_chat_total_last_month.loc[: , ['일반채팅 수', '답장 수', '전체공개 수', '좋아요 수', '컨텐츠 수']]


celeb_chat_total_last_month

In [ ]:
celeb_chat_total_last_month.info()

#### 2.3. Rate 

In [ ]:
# calculate increase, decrease rate

def calculate_change_rate(pre_val, curr_val):

  change_rate = (curr_val - pre_val) / pre_val * 100

  return  change_rate 

In [ ]:
celeb_chat_total_this_month['일반채팅_증감율'] = calculate_change_rate(celeb_chat_total_last_month['일반채팅 수'], celeb_chat_total_this_month['일반채팅 수'])
celeb_chat_total_this_month['답장_증감율'] = calculate_change_rate(celeb_chat_total_last_month['답장 수'], celeb_chat_total_this_month['답장 수'])
celeb_chat_total_this_month['전체공개_증감율'] = calculate_change_rate(celeb_chat_total_last_month['전체공개 수'], celeb_chat_total_this_month['전체공개 수'])
celeb_chat_total_this_month['좋아요_증감율'] = calculate_change_rate(celeb_chat_total_last_month['좋아요 수'], celeb_chat_total_this_month['좋아요 수'])
celeb_chat_total_this_month['컨텐츠_증감율'] = calculate_change_rate(celeb_chat_total_last_month['컨텐츠 수'], celeb_chat_total_this_month['컨텐츠 수'])

celeb_chat_total_this_month = celeb_chat_total_this_month.fillna(0)


celeb_chat_total_this_month

In [ ]:
celeb_chat_total_this_month.info()

#### 2.4. Calculation of mean ratio

In [ ]:
#Calculate mean 
#팬심 M 평균을 계산하는 코드 
#멀티인덱스는 알아서 맞춰줍니다 

import numpy as np 

def calculate_mean(val):

   avg = int(np.mean(val))

   return avg 


# Calcutate Celeb's activation of mean ratio
# 팬심 평균 대비 셀럽 활동 지표를 계산하는 코드 
# (대상값 - 평균값 / 평균값) *100


def calculate_percentage(val, mean):

  output =  ((val - mean)/ mean) *100

  return output


In [ ]:
#celeb_chat_total_this_month['일반채팅_평균'] = calculate_mean(celeb_chat_total_this_month['일반채팅 수'])
celeb_chat_total_this_month['답장_평균'] = calculate_mean(celeb_chat_total_this_month['답장 수'])
celeb_chat_total_this_month['전체공개_평균'] = calculate_mean(celeb_chat_total_this_month['전체공개 수'])
celeb_chat_total_this_month['좋아요_평균'] = calculate_mean(celeb_chat_total_this_month['좋아요 수'])
celeb_chat_total_this_month['컨텐츠_평균'] = calculate_mean(celeb_chat_total_this_month['컨텐츠 수'])

celeb_chat_total_this_month['일반채팅_평균대비'] = calculate_percentage(celeb_chat_total_this_month['일반채팅 수'],celeb_chat_total_this_month['일반채팅_평균'])
celeb_chat_total_this_month['답장_평균대비'] = calculate_percentage(celeb_chat_total_this_month['답장 수'], celeb_chat_total_this_month['답장_평균'])
celeb_chat_total_this_month['전체공개_평균대비'] = calculate_percentage(celeb_chat_total_this_month['전체공개 수'], celeb_chat_total_this_month['전체공개_평균'])
celeb_chat_total_this_month['좋아요_평균대비'] = calculate_percentage(celeb_chat_total_this_month['좋아요 수'], celeb_chat_total_this_month['좋아요_평균'])
celeb_chat_total_this_month['컨텐츠_평균대비'] = calculate_percentage(celeb_chat_total_this_month['컨텐츠 수'], celeb_chat_total_this_month['컨텐츠_평균'])


celeb_chat_total_this_month = celeb_chat_total_this_month.fillna(0)


celeb_chat_total_this_month

#3. Fan action data

#### 3.1. Fan Chat data 

In [ ]:
%%bigquery fan_chat_this_month --project  project name 

SELECT 
  chat_room_id, 
  chat_celeb_nickname,
  user_nickname, 
  count(message_length) AS message, 
  SUM(IFNULL(CAST(product_price_string AS INT64), 0)) AS thanku,
  SUM(CASE WHEN chat_room_btn_icon_name = 'Annouce' THEN 1 ELSE 0 END) AS Annouce,
  SUM(CASE WHEN chat_room_btn_icon_name = 'Copy' THEN 1 ELSE 0 END) AS Copy,
  SUM(CASE WHEN chat_room_btn_icon_name = 'Delete' THEN 1 ELSE 0 END) AS Delete,
  SUM(CASE WHEN chat_room_btn_icon_name = 'Image' THEN 1 ELSE 0 END) AS Image,
  SUM(CASE WHEN chat_room_btn_icon_name = 'Like' THEN 1 ELSE 0 END) AS Liked,
  SUM(CASE WHEN chat_room_btn_icon_name = 'Record' THEN 1 ELSE 0 END) AS Record,
  SUM(CASE WHEN chat_room_btn_icon_name = 'Reply' THEN 1 ELSE 0 END) AS Reply,
  SUM(CASE WHEN chat_room_btn_icon_name = 'Report_Message' THEN 1 ELSE 0 END) AS Report_Message,
  SUM(CASE WHEN chat_room_btn_icon_name = 'Share' THEN 1 ELSE 0 END) AS Share,
  SUM(CASE WHEN chat_room_btn_icon_name = 'Translate' THEN 1 ELSE 0 END) AS Translate,
  SUM(CASE WHEN chat_room_btn_icon_name = 'Video' THEN 1 ELSE 0 END) AS Video,
  SUM(SAFE_CAST(product_price_string AS FLOAT64)) as thanks_total, #non-numeric value 를 sum 하기 위해 
  COUNT(product_price_string) as thanks_count


FROM  `project name.mixpanel_test.mp_master_event` 

WHERE (time BETWEEN '2023-03-01 00:00:00' AND '2023-03-31 23:59:59')
AND (user_nickname != chat_celeb_nickname)
AND (chat_room_id != '')

GROUP BY 
  chat_room_id, 
  chat_celeb_nickname, 
  user_nickname

ORDER BY 
  chat_room_id, 
  chat_celeb_nickname, 
  user_nickname

In [ ]:
fan_chat_this_month.rename(columns={'message':'일반채팅 수'}, inplace = True)
fan_chat_this_month.rename(columns={'Liked':'좋아요 수'}, inplace = True)
fan_chat_this_month.rename(columns={'thanks_total':'땡큐스티커 총 금액(샷)'}, inplace = True)
fan_chat_this_month.rename(columns={'thanks_count':'땡큐스티커 발송 횟수'}, inplace = True)
fan_chat_this_month = fan_chat_this_month.loc[: , ['chat_room_id', 'chat_celeb_nickname', 'user_nickname','일반채팅 수', '좋아요 수', '땡큐스티커 총 금액(샷)','땡큐스티커 발송 횟수' ]]
fan_chat_this_month = fan_chat_this_month.fillna(0)

#그룹바이 안 하면 user_nickname 살아있음
fan_chat_this_month = fan_chat_this_month.groupby(['chat_room_id','chat_celeb_nickname']).sum()


fan_chat_this_month

#### 3.2. Fan act Last month 

In [ ]:
%%bigquery fan_chat_last_month --project  project name 

SELECT 
  chat_room_id, 
  chat_celeb_nickname,
  user_nickname, 
  count(message_length) AS message, 
  SUM(IFNULL(CAST(product_price_string AS INT64), 0)) AS thanku,
  SUM(CASE WHEN chat_room_btn_icon_name = 'Annouce' THEN 1 ELSE 0 END) AS Annouce,
  SUM(CASE WHEN chat_room_btn_icon_name = 'Copy' THEN 1 ELSE 0 END) AS Copy,
  SUM(CASE WHEN chat_room_btn_icon_name = 'Delete' THEN 1 ELSE 0 END) AS Delete,
  SUM(CASE WHEN chat_room_btn_icon_name = 'Image' THEN 1 ELSE 0 END) AS Image,
  SUM(CASE WHEN chat_room_btn_icon_name = 'Like' THEN 1 ELSE 0 END) AS Liked,
  SUM(CASE WHEN chat_room_btn_icon_name = 'Record' THEN 1 ELSE 0 END) AS Record,
  SUM(CASE WHEN chat_room_btn_icon_name = 'Reply' THEN 1 ELSE 0 END) AS Reply,
  SUM(CASE WHEN chat_room_btn_icon_name = 'Report_Message' THEN 1 ELSE 0 END) AS Report_Message,
  SUM(CASE WHEN chat_room_btn_icon_name = 'Share' THEN 1 ELSE 0 END) AS Share,
  SUM(CASE WHEN chat_room_btn_icon_name = 'Translate' THEN 1 ELSE 0 END) AS Translate,
  SUM(CASE WHEN chat_room_btn_icon_name = 'Video' THEN 1 ELSE 0 END) AS Video,
  SUM(SAFE_CAST(product_price_string AS FLOAT64)) as thanks_total, #non-numeric value 를 sum 하기 위해 
  COUNT(product_price_string) as thanks_count


FROM  `project name.mixpanel_test.mp_master_event` 

WHERE (time BETWEEN '2023-02-01 00:00:00' AND '2023-02-28 23:59:59')
AND (user_nickname != chat_celeb_nickname)
AND (chat_room_id != '')

GROUP BY 
  chat_room_id, 
  chat_celeb_nickname, 
  user_nickname

ORDER BY 
  chat_room_id, 
  chat_celeb_nickname, 
  user_nickname

In [ ]:
fan_chat_last_month.rename(columns={'message':'일반채팅 수'}, inplace = True)
fan_chat_last_month.rename(columns={'Liked':'좋아요 수'}, inplace = True)
fan_chat_last_month.rename(columns={'thanks_total':'땡큐스티커 총 금액(샷)'}, inplace = True)
fan_chat_last_month.rename(columns={'thanks_count':'땡큐스티커 발송 횟수'}, inplace = True)
fan_chat_last_month = fan_chat_last_month.loc[: , ['chat_room_id', 'chat_celeb_nickname', 'user_nickname','일반채팅 수', '좋아요 수', '땡큐스티커 총 금액(샷)','땡큐스티커 발송 횟수' ]]
fan_chat_last_month = fan_chat_last_month.fillna(0)

#그룹바이 안 하면 user_nickname 살아있음
fan_chat_last_month = fan_chat_last_month.groupby(['chat_room_id','chat_celeb_nickname']).sum()

fan_chat_last_month

#### 3.3. Increase / decrease rate

In [ ]:
def calculate_change_rate(pre_val, curr_val):

  change_rate = (curr_val - pre_val) / pre_val * 100

  return  change_rate 


fan_chat_this_month['일반채팅_증감율'] = calculate_change_rate(fan_chat_last_month['일반채팅 수'], fan_chat_this_month['일반채팅 수'])
fan_chat_this_month['좋아요_증감율'] = calculate_change_rate(fan_chat_last_month['좋아요 수'], fan_chat_this_month['좋아요 수'])
fan_chat_this_month['땡큐스티커_총금액_증감율'] = calculate_change_rate(fan_chat_last_month['땡큐스티커 총 금액(샷)'], fan_chat_this_month['땡큐스티커 총 금액(샷)'])
fan_chat_this_month['땡큐스티커_횟수_증감율'] = calculate_change_rate(fan_chat_last_month['땡큐스티커 발송 횟수'], fan_chat_this_month['땡큐스티커 발송 횟수'])

fan_chat_this_month.fillna(0,inplace=True)


fan_chat_this_month

####3.4.  Calculation of mean ratio

In [ ]:
import numpy as np

fan_chat_this_month['일반채팅_평균'] = calculate_mean(fan_chat_this_month['일반채팅 수'])
fan_chat_this_month['좋아요_평균'] = calculate_mean(fan_chat_this_month['좋아요 수'])
fan_chat_this_month['땡스총액_평균'] = calculate_mean(fan_chat_this_month['땡큐스티커 총 금액(샷)'])
fan_chat_this_month['땡스횟수_평균'] = calculate_mean(fan_chat_this_month['땡큐스티커 발송 횟수'])

fan_chat_this_month['일반채팅_평균대비'] = calculate_percentage(fan_chat_this_month['일반채팅 수'], fan_chat_this_month['일반채팅_평균'])
fan_chat_this_month['좋아요_평균대비'] = calculate_percentage(fan_chat_this_month['좋아요 수'], fan_chat_this_month['좋아요_평균'])
fan_chat_this_month['땡스총액_평균대비'] = calculate_percentage(fan_chat_this_month['땡큐스티커 총 금액(샷)'], fan_chat_this_month['땡스총액_평균'])
fan_chat_this_month['땡스횟수_평균대비'] = calculate_percentage(fan_chat_this_month['땡큐스티커 발송 횟수'], fan_chat_this_month['땡스횟수_평균'])

fan_chat_this_month.replace([np.inf, -np.inf], 0, inplace=True)
fan_chat_this_month.replace(np.nan, 0, inplace=True)
fan_chat_this_month.fillna(0, inplace=True)

fan_chat_this_month

In [ ]:
celeb_chat_total_this_month

#4. Automated Comment (자동 코멘트) 

In [ ]:
cols = ['일반채팅_증감율', '답장_증감율', '전체공개_증감율', '좋아요_증감율', '컨텐츠_증감율']

#Count the number of negative values 
celeb_chat_total_this_month['negative_count'] = (celeb_chat_total_this_month[cols] <= 0).sum(axis=1)

# Minimum value
celeb_chat_total_this_month['blue_light1'] = celeb_chat_total_this_month[cols].apply(
    lambda row: sorted(row.items(), key=lambda x: float('inf') if np.isinf(x[1]) else x[1])[0][0], axis=1).str.split('_').str[0]


# Maximum value
celeb_chat_total_this_month['red_light1'] = celeb_chat_total_this_month[cols].apply(
    lambda row: sorted(row.items(), key=lambda x: -np.inf if np.isinf(x[1]) else x[1])[-1][0],
    axis=1
).str.split('_').str[0]


pd.set_option('display.max_columns', None)
celeb_chat_total_this_month.tail(5)

In [ ]:
#Specify comments differently by setting conditions to the number of negative items specified earlier. Comments have different intentions due to motivation/compliments, etc
#앞서 지정한 음수항목 갯수로 조건을 걸어 코멘트를 다르게 지정함. 동기부여 / 칭찬 등으로 코멘트의 의도가 다름

def automate_comments(df, month):
     
    #자동코멘트가 들어갈 리스트 #List of autocoments
    comments = []

    for celeb_name, group in df.groupby(level=0):

        
        negative_count = group['negative_count'].sum()

        
        if negative_count > 4:

            
            blue1 = group['blue_light1'].iloc[0]
            comments.append(f"{celeb_name} …자니..? {month}월 팬심활동이 잠들어가고있어요. 이번달에는 {blue1}에 더 신경써 볼까요? ") #comments 

        elif negative_count > 2:

            red1 = group['red_light1'].iloc[0]
            blue1 = group['blue_light1'].iloc[0]
            comments.append(f"{month} 월에는 {red1}에 최선을 다하셨군요!  다음에는 {blue1}도 해보시는게 어때요? 나의 작은 아기 고양이😉")  #comments 

        elif negative_count > 1: 

            red1 = group['red_light1'].iloc[0]

            
            comments.append(f"{month}월에는 {red1}에 신경써서서 팬들의 마음을 사로잡았어요. {celeb_name}님 폼 미쳤다")  #comments 

        else:
            comments.append(f"{month}월에는 팬을 정말 열심히 챙겼어요.{celeb_name}님 좀 치시는군요? 박수를? 👏👏👏")  #comments 
            
    return comments


celeb_chat_total_this_month['comments'] = automate_comments(celeb_chat_total_this_month, 3)

In [ ]:
celeb_chat_total_this_month['negative_count'].value_counts()

In [ ]:
#check 
celeb_chat_total_this_month[celeb_chat_total_this_month['negative_count'] == 3]

In [ ]:
#check 
celeb_chat_total_this_month[celeb_chat_total_this_month['negative_count'] == 2]

#4. Save

In [ ]:
import pandas as pd

fan_chat_this_month.to_csv('./Fancare_report_page0_2_fan.csv')
celeb_chat_total_this_month.to_csv('./Fancare_report_page0_2_celeb.csv')